In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input, Concatenate
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Data Loading with Augmentation
base_path = '../Database/'
data, labels = [], []

data_gen = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
    brightness_range=[0.8, 1.2], fill_mode='nearest'
)

def load_images_from_folder(folder, label):
    folder_path = os.path.join(base_path, folder)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, (128, 128))
        data.append(img)
        labels.append(label)

load_images_from_folder("Normal", "Normal")
for folder in ["Lung_Opacity", "Viral Pneumonia"]:
    load_images_from_folder(folder, "Lung_Disease")

data = np.array(data).astype('float32') / 255.0
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    data, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded
)

# Extract HOG Features
def extract_hog_features(images):
    return np.array([hog(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), visualize=False) for img in images])

X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

# Handle Imbalance Using SMOTE
# Handle Imbalance Using SMOTE (Apply to Combined Features)
X_train_combined = np.hstack((X_train.reshape(X_train.shape[0], -1), X_train_hog))

# Apply SMOTE to the Combined Features
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_combined, y_train)

# Split the balanced data back into CNN and HOG parts
X_train_cnn_balanced = X_train_balanced[:, :X_train.size // X_train.shape[0]].reshape(-1, 128, 128, 1)
X_train_hog_balanced = X_train_balanced[:, X_train.size // X_train.shape[0]:]

# Standardize Features
scaler = StandardScaler()
X_train_hog_balanced = scaler.fit_transform(X_train_hog_balanced)
X_test_hog = scaler.transform(X_test_hog)


# Feature Scaling
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_test_hog = scaler.transform(X_test_hog)

# Build DenseNet Model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model layers

# Feature extraction from DenseNet
cnn_input = Input(shape=(128, 128, 3))
cnn_features = base_model(cnn_input, training=False)
cnn_features = Flatten()(cnn_features)
cnn_features = Dense(256, activation='relu')(cnn_features)
cnn_features = BatchNormalization()(cnn_features)
cnn_features = Dropout(0.5)(cnn_features)

# HOG Input Layer
hog_input = Input(shape=(X_train_hog.shape[1],))
hog_features = Dense(128, activation='relu')(hog_input)
hog_features = BatchNormalization()(hog_features)
hog_features = Dropout(0.5)(hog_features)

# Concatenate CNN and HOG Features
combined_features = Concatenate()([cnn_features, hog_features])
final_dense = Dense(64, activation='relu')(combined_features)
final_dense = BatchNormalization()(final_dense)
final_dense = Dropout(0.4)(final_dense)
output = Dense(1, activation='sigmoid')(final_dense)

# Create Model
model = Model(inputs=[cnn_input, hog_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy', metrics=['accuracy'])

# Learning Rate Scheduler & Early Stopping
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

# Train the model
history = model.fit([X_train, X_train_hog], y_train, validation_data=([X_test, X_test_hog], y_test),
                    epochs=50, callbacks=[lr_scheduler, early_stopping], batch_size=32)

# Evaluation
y_pred = (model.predict([X_test, X_test_hog]) > 0.55).astype("int32")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


Epoch 1/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.7693 - loss: 0.5098 - val_accuracy: 0.8316 - val_loss: 2.5639 - learning_rate: 5.0000e-04
Epoch 2/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 219s 3s/step - accuracy: 0.8853 - loss: 0.2931 - val_accuracy: 0.8632 - val_loss: 1.5110 - learning_rate: 5.0000e-04
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.9155 - loss: 0.2136 - val_accuracy: 0.8692 - val_loss: 1.7066 - learning_rate: 5.0000e-04
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.9388 - loss: 0.1571 - val_accuracy: 0.8541 - val_loss: 2.1503 - learning_rate: 5.0000e-04
Epoch 5/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.9540 - loss: 0.1248 - val_accuracy: 0.8511 - val_loss: 2.3973 - learning_rate: 5.0000e-04
Epoch 6/50
 6/83 ━━━━━━━━━━━━━━━━━━━━ 1:48 1s/step - accuracy: 0.9737 - loss: 0.0792